<a href="https://colab.research.google.com/github/abhi9rocks/DeepStuff/blob/master/rnn1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd '/content/gdrive/My Drive/RNN/'

/content/gdrive/My Drive/RNN


In [3]:
!wget "https://github.com/crwong/cs224u-project/blob/master/data/sentiment/training.1600000.processed.noemoticon.csv"

--2021-03-08 12:31:59--  https://github.com/crwong/cs224u-project/blob/master/data/sentiment/training.1600000.processed.noemoticon.csv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘training.1600000.processed.noemoticon.csv.1’

training.1600000.pr     [ <=>                ] 102.05K  --.-KB/s    in 0.04s   

2021-03-08 12:31:59 (2.69 MB/s) - ‘training.1600000.processed.noemoticon.csv.1’ saved [104499]



In [4]:
pip install keras==2.1.6

In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers.convolutional import Conv1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import spacy
from pylab import *
nlp=spacy.load("en")

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [9]:
ls


training.1600000.processed.noemoticon.csv


In [11]:
#load the dataset
train=pd.read_csv("training.1600000.processed.noemoticon.csv" , encoding= "latin-1", error_bad_lines=False)
Y_train = train[train.columns[0]]
X_train = train[train.columns[0]]

b'Skipping line 53: expected 1 fields, saw 2\nSkipping line 58: expected 1 fields, saw 2\nSkipping line 59: expected 1 fields, saw 2\nSkipping line 73: expected 1 fields, saw 2\nSkipping line 92: expected 1 fields, saw 193\nSkipping line 93: expected 1 fields, saw 2\nSkipping line 155: expected 1 fields, saw 3\nSkipping line 167: expected 1 fields, saw 6\nSkipping line 168: expected 1 fields, saw 7\nSkipping line 193: expected 1 fields, saw 4\nSkipping line 197: expected 1 fields, saw 2\nSkipping line 210: expected 1 fields, saw 2\nSkipping line 211: expected 1 fields, saw 2\nSkipping line 217: expected 1 fields, saw 2\nSkipping line 220: expected 1 fields, saw 2\nSkipping line 228: expected 1 fields, saw 4\nSkipping line 234: expected 1 fields, saw 2\nSkipping line 239: expected 1 fields, saw 2\nSkipping line 240: expected 1 fields, saw 2\nSkipping line 241: expected 1 fields, saw 2\nSkipping line 242: expected 1 fields, saw 2\nSkipping line 243: expected 1 fields, saw 2\nSkipping lin

In [12]:

# split the data into test and train
from sklearn.model_selection import train_test_split
trainset1x, trainset2x, trainset1y, trainset2y = train_test_split(X_train.values, Y_train.values, test_size=0.02,random_state=42 )
trainset2y=pd.get_dummies(trainset2y)

In [15]:
# function to remove stopwords
def stopwords(sentence):
   new=[]
   sentence=nlp(sentence)
   for w in sentence:
      if (w.is_stop == False) & (w.pos_ !="PUNCT"):
        new.append(w.string.strip())
        c=" ".join(str(x) for x in new)
      return c

In [17]:
# function to lemmatize the tweets
def lemmatize(sentence):
    sentence=nlp(sentence)
    str=""
    for w in sentence:
        str+=" "+w.lemma_
    return nlp(str)

In [28]:
#loading the glove model
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = [float(val) for val in splitLine[1:]]
        model[word] = embedding
    print ("Done."),len(model),(" words loaded!")
    return model

In [25]:
!wget "https://www.kaggle.com/fullmetal26/glovetwitter27b100dtxt"

--2021-03-08 12:46:09--  https://www.kaggle.com/fullmetal26/glovetwitter27b100dtxt
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘glovetwitter27b100dtxt’

glovetwitter27b100d     [ <=>                ]  29.55K  --.-KB/s    in 0.003s  

2021-03-08 12:46:09 (11.3 MB/s) - ‘glovetwitter27b100dtxt’ saved [30261]



In [26]:
ls

glovetwitter27b100dtxt  training.1600000.processed.noemoticon.csv


In [29]:
# save the glove model
model=loadGloveModel("glove.twitter.27B.200d.txt")

Loading Glove Model


FileNotFoundError: ignored

In [31]:
#vectorising the sentences
def sent_vectorizer(sent, model):
    sent_vec = np.zeros(200)
    numw = 0
    for w in sent.split():
        try:
            sent_vec = np.add(sent_vec, model[str(w)])
            numw+=1
        except:
            pass
    return sent_vec

In [32]:
#obtain a clean vector
cleanvector=[]
for i in range(trainset2x.shape[0]):
    document=trainset2x[i]
    document=document.lower()
    document=lemmatize(document)
    document=str(document)
    cleanvector.append(sent_vectorizer(document,model))

NameError: ignored